# M-I coupling with Lompe

M-I coupling: AMPS + Hardy / EUV conductance?

In [ ]:
%matplotlib inline
import lompe
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import pandas as pd
import apexpy
from scipy.interpolate import griddata
from scipy.ndimage import median_filter

Re = 6371.2e3                   # Earth's radius [m]
HI = 110e3                      # Ionospheric altitude [m]
stencil_size = 1                # size of stencil for differentiation 
time = dt.datetime(2005, 9, 10) # used for coordinate conversion, map plotting, and solar zenith angles (sza)

#### Cubed sphere grid

In [ ]:
L, W, Lres, Wres = 18000e3, 18000e3, 80e3, 80e3  # m
projection = lompe.cs.CSprojection(position = (0, 90), orientation = 0)        # east, west oriented xi-axis
grid = lompe.cs.CSgrid(projection, L, W, Lres, Wres, R = Re+HI) # set to ionospheric altitude
SKIP = 20 # how many grid points to skip when smoothing 

##### Ionospheric conductances

In [ ]:
Kp   = 4                        # for Hardy model
F107 = 100                      # sfu, for EUV conductance
refh = HI*1e-3                  # altitude for coodinate conversion [km]

# functions for conductances to be passed to the Lompe model
SH = lambda lon = grid.lon, lat = grid.lat: lompe.conductance.hardy_EUV(lon, lat, Kp, time, 'hall')
SP = lambda lon = grid.lon, lat = grid.lat: lompe.conductance.hardy_EUV(lon, lat, Kp, time, 'pedersen')

#### FACs from the AMPS model

Average Magnetic field and Polar current System (AMPS) model ([Laundal et al., 2018](https://doi.org/10.1029/2018JA025387))

In [ ]:
# AMPS FACs


NB: Input data for Lompe of the `fac` datatype need to be defined on the entire grid. This is done here, as the FACs are defined in the whole region, and we have interpolated the data to the grid.

In [ ]:
# field-aligned current
FACdata = lompe.Data(facinterp.flatten(), np.vstack((grid.lon.flatten(), grid.lat.flatten())), datatype = 'fac')

##### Construct the Lompe model
Now, we can initialize a Lompe model object, add the input data, and run the inversion to get the model vector, $\mathbf{m}$, that can be used to obtain other ionospheric parameters in the region. An explanation of the inversion technique and regularization parameters can be found in Section 3.3 of [Laundal et al. (2022)](https://doi.org/10.1029/2022JA030356).

In [ ]:
# initialize model
model = lompe.Emodel(grid, Hall_Pedersen_conductance = (SH, SP))

# add input data
model.add_data(FACdata)

# Run inversion. l1 and l2 are regularization parameters that control the damping of 
# 1) model norm, and 2) gradient of SECS amplitudes (charges) in magnetic eastward direction
model.run_inversion(l1 = 0.01, l2 = 0.2);

#### Plot and save

In [ ]:
fig = lompe.lompeplot(model, include_data=False, time=time, apex=a, savekw = {'fname':'./output_figures/MIwLompe.pdf'})
# also save png
fig.savefig('./output_figures/MIwLompe.png')

The top row shows, from left to right: 
- Convection flow field and electric potential contours
- horizontal magnetic field disturbances 110 km above the ionosphere (grid altitude) as black arrows and radial current density as color contours
- horizontal ground magnetic field perturbations as black arrows and radial magnetic field perturbations as color contours
- a map that shows the grid’s position and orientation with respect to apex magnetic latitude and local time.

The bottom row shows, from left to right: 
- Pedersen conductance
- Hall conductance
- horizontal height-integrated ionospheric currents based on Lompe output
- color scale / vector scales